# Trackmania Project : Optimizing checkpoints order on E02

## Le projet : 
_______

Le but de notre projet était de répondre à la problématique : Quelle est la meilleure façon de parcourir les différents checkpoints d'un circuit pour le terminer le plsu rapidement possible ? 

Nous nous intéressons ici au cas particulier du circuit E02 du jeu vidéo TrackMania Nations Forever

![alt text](hqdefault.jpg)

___
    
Nous nous sommes intéressés à ce circuit en particulier puisque c'est le circuit possèdant les plus de voies alternatives et de checkpoints à valider de la série de jeu TrackMania.

Pour le petit historique, plusieurs millions de joueurs essayent tant bien que mal d'inscrire leur temps dans le haut du classement de chaque circuit. De 2008 à 2019, le record de E02 n'a cessé de baisser jusqu'à atteindre les 3:48.59, record réalisé par Wirtual.

Le niveau de jeu des joueurs experts étant assez homogène, les places dans le top 100 se jouent à quelques centièmes de secondes.

Il y a de cela 10 mois, Wirtual bat son propre record du monde de plus de 14 secondes. 
Pour réaliser cet exploit, Wirtual a utilisé de noumbreux "cuts" (passage d'une partie à une autre du circuit, non prévu par les créateurs). Tous ces cuts étaient déjà largement connus parmis les meilleurs joueurs, mais il était assez compliqué de savoir si ils pouvaient ou non faire gagner du temps, et si oui dans quel ordre les prendre. 
Le nombre de cuts possibles étant trop élevé pour une recherche exhaustive, sans compter sur la difficulté de réalisation de certains d'entre eux, Wirtual décida de chronométrer tous les passages possibles et d'exprimer ce problème comme un problème de voyageur de commerce (Traveling Salesman Problem).

> Le problème du voyageur de commerce, ou problème du commis voyageur, est un problème d'optimisation qui, étant donné une liste de villes, et des distances entre toutes les paires de villes, détermine un plus court chemin qui visite chaque ville une et une seule fois et qui termine dans la ville de départ. 

Ici, les villes constitueront les checkpoints et la distance sera définie comme le temps nécessaire pour aller d'un checkpoint à un autre. Nous pouvons noter qu'il s'agit d'un problème dit TSP asymétrique puisque il n'est pas toujours possible de faire la jonction entre deux checkpoints dans les deux sens.

Notre objectif dans ce projet est donc d'essayer de réaliser la même étude, en s'appuyant sur différentes méthodes de résolution, à savoir : l'algorithme de Little (Branch and Bound) et une modélisation du problème sous forme de Programmation Linéaire en Nombres Entiers

## L'avancement du projet
____

Nous avons décider d'organiser notre travail dans l'odre suivant : 
* **Obternir un version fonctionnelle avec l'algorithe de Little, sur l'exemple du polycopié**
* **Réussir à importer nos données depuis un fichier csv pour faciliter l'importation des temps**
* **Tester l'algorithme de Little sur un circuit de notre cru (Avec peu de checkpoints)**
* **Obtenir une version fonctionnelle de la version PLNE (Miller-Tucker-Zemlin)**
* Obtenir une version fonctionnelle de la version PLNE (Dantzig-Fulkerson-Johnson)
* Récupérer les données de temps du circuit E02
* Importer les données de temps du circuit E02
* Comparer les temps d'éxécution et les résulats de nos différentes approches sur les données de E02
* Comparaison générale des 3 approches
* Optionnel : Test des algorithmes génétiques / fourmillière de résolution de TSP, même si la taille de notre problème ne s'y prête pas vraiment
* Tentative de record du monde par Hugo

Comme vous pourrez être amené à le voir ci-dessous, une majorité des étapes les plus difficiles a déja été réalisée. L'algorithme de Little et la formulation PLNE (M-T-Z) semblent parfaitement fonctionner sur les différents exemples que nous leur avons passés.

Nous avons choisi l'approche "dictionnaire" plutôt que l'approche matricielle pour la description puLp du problème, puisque contrairement à un problème TSP plus "classique", une majorité des trajets ne peut pas être réalisé dans un temps raisonnable. Une modélisation matricielle aurait défini beaucoup de contraintes inutiles et complexifié notre problème.

Finalement, nous allons passer les prochains jours à nous atteler à la modélisation (D-F_J) du modèle PLNE et à la comparaison de nos différentes méthodes.

Le fait que la majorité des trajets ne soit pas possible semble "diminuer la complexité" de l'algorithme de Little et nous avons été agréablement surpris du peu de braches explorées pour la résolution de notre exemple jouet à 8 checkpoints. Alors que l'algorithme est censé exploser en complexité bien avant la marque des 26 villes, nous sommes pressés de voir si l'algorithme parviendra tout de même à trouver meilleur chemin

Comme écrit dans la liste des tâches, nous essaierons également d'effectuer quelques tests d'algorithmes de résolution génétiques, bien que notre problème soit de taille assez faible pour être résolu de manière exacte en un temps convenable

# Branch and Bound Algorithm (Little)

## Dependencies

In [1]:
import time
import numpy as np
from treelib import Node, Tree

### Defining functions

In [2]:
def matrixReduction(matrix):
    red_matrix = np.copy(matrix)
    n = red_matrix.shape[0]
    evaluation = 0
    
    for j in range(n):
        if(np.min(red_matrix[:,j])!= np.inf):
            evaluation += np.min(red_matrix[:,j])
            red_matrix[:,j] = red_matrix[:,j] - np.min(red_matrix[:,j])

    for i in range(n):
        if(np.min(red_matrix[i,:])!= np.inf):
            evaluation += np.min(red_matrix[i,:])
            red_matrix[i,:] = red_matrix[i,:] - np.min(red_matrix[i,:])
            
    return (red_matrix, evaluation) 

In [3]:
def calculerRegret(i,j, matrix):
    mat_cout = np.copy(matrix)
    mat_cout[i,j] = np.inf
    cout = np.min(mat_cout[i,:])
    cout += np.min(mat_cout[:,j])
    return cout

In [4]:
def visitNode(n, matrix, e, unavailable,parent_id):
    
    global maxi
    global tree
    global node_id
    root = nodes[0]
    s = tree.get_node(parent_id).tag #gets the name of the parent node
    
    #print('current:{} - e:{} -unavailable:{} -maxi:{}'.format(villes[n],e,unavailable,maxi))
    #print(matrix)
    
    if(e<=maxi):
        
        if(n != 0 or s[:len(root)] == root): #check if the parent node is the root
            
            availableNodes = np.where(matrix[n,:]!=np.inf)[0]
            regrets = {}
            change = False
            
            for node in availableNodes:
                if(node!=unavailable):
                    regrets[node] = calculerRegret(n,node, matrix)
                    change = True
            
            if(change):
                
                maxRegret = int(max(regrets, key=regrets.get)) #Retourne la clé de la valeur max
                
                new_matrix = np.copy(matrix)
                new_matrix[n,:] = np.inf
                new_matrix[:,maxRegret] = np.inf
                new_matrix[maxRegret,n] = np.inf
                new_matrix_2 = np.copy(matrix)
                new_matrix_2[n,maxRegret] = np.inf
                
                (new_matrix, coutevaluation) = matrixReduction(new_matrix)
                (new_matrix_2, coutevaluation2) = matrixReduction(new_matrix_2)
                
                localid = int(node_id)
                #________V__________
                e_v = e + coutevaluation
                
                #Créer l'arbre, les noeuds, tout ça
                node_id += 1
                tree.create_node(nodes[maxRegret]+' '+str(e_v), node_id, parent=localid, data=e_v)
                                
                visitNode(maxRegret,new_matrix,e_v,None,localid)
                
                #______Vbarre_______
                e_v_barre = e + regrets[maxRegret]
                node_id += 1
                tree.create_node(nodes[n]+'!'+nodes[maxRegret]+' '+str(e_v_barre), node_id, parent=localid, data=e_v_barre)
                          
                visitNode(n,new_matrix_2, e_v_barre,None,localid)

        else:
            #print('Le maxi va changer : ni: {} - pi: {}'.format(node_id,parent_id))
            maxi = min(maxi,e)
            #print('Maxi : {}'.format(maxi),'\n')

In [5]:
#Fonction appelant l'algorithe de résolution de Little
def Little(matrix, racine):
    
    global maxi
    global tree
    global node_id
    localid = int(node_id)
    
    n=racine
    (matrix, e) = matrixReduction(matrix)
    unavailable = None #Réduire la matrice et calculer E
    
    tree.create_node(nodes[n], node_id)
    
    visitNode(n,matrix, e,None, localid) #Visiter la racine

In [6]:
#Affichage du meilleur chemin
def Path(tree):
    
    minimum = np.inf
    min_node = None
    for node in tree.all_nodes_itr():
        if(node.tag[:len(nodes[0])]==nodes[0] and node.data != None and node.tag[len(nodes[0])]!="!"):
            if(node.data <= minimum):
                minimum = node.data
                min_node = node


    depth = tree.depth(min_node)
    node = min_node
    path = [min_node.tag.split(' ')[0]]
    #print('min node :', min_node)
    
    for i in range(depth):
        node = tree.parent(node.identifier)
        #print(node)
        if("!" not in node.tag):
            path.append(node.tag.split(' ')[0])
    path.reverse()
    return path

## Testing

### >Textbook example

In [7]:
maxi = np.inf
costs = np.array([[maxi,780,320,580],
                 [780,maxi,700,460],
                 [320,700,maxi,380],
                  [580,460,380,maxi]])
nodes = ['Bordeaux','Lyon','Nantes','Paris']

In [8]:
tree = Tree()
node_id = 1

%time Little(costs,0)

Wall time: 3 ms


In [9]:
Path(tree)

['Bordeaux', 'Lyon', 'Paris', 'Nantes', 'Bordeaux']

### >Toy example

___

<img src="toyexample2.jpg" alt="drawing" width="250"/>

In [10]:
costs = np.array([[maxi,10,maxi,maxi,maxi,maxi,maxi,maxi],
                 [10,maxi,10,25,maxi,maxi,maxi,maxi],
                 [maxi,10,maxi,15,5,7,maxi,maxi],
                 [maxi,25,15,maxi,5,maxi,4,maxi],
                 [maxi,maxi,5,5,maxi,15,20,maxi],
                 [maxi,maxi,7,maxi,15,maxi,5,6],
                 [maxi,maxi,maxi,4,20,5,maxi,10],
                 [0,maxi,maxi,maxi,maxi,maxi,maxi,maxi]])
nodes = [str(i+1) for i in range(8)]

In [11]:
tree = Tree()
node_id = 1

%time Little(costs,0)

Wall time: 4 ms


In [12]:
tree.show()

1
├── 1!2 1978.0
└── 2 44.0
    ├── 2!3 60.0
    └── 3 45.0
        ├── 3!5 47.0
        └── 5 45.0
            ├── 4 45.0
            │   ├── 4!7 1980.0
            │   └── 7 45.0
            │       ├── 6 45.0
            │       │   ├── 6!8 3919.0
            │       │   └── 8 45.0
            │       │       ├── 1 45.0
            │       │       └── 8!1 inf
            │       └── 7!6 1984.0
            └── 5!4 54.0



In [13]:
Path(tree)

['1', '2', '3', '5', '4', '7', '6', '8', '1']

## From csv

In [14]:
costs = np.genfromtxt('villes.csv', delimiter=';')
costs = np.array(costs, dtype=float)

In [15]:
costs = np.nan_to_num(costs, nan=np.inf)
nodes = ['Bordeaux','Lyon','Nantes','Paris']

In [16]:
costs

array([[ inf, 780., 320., 580.],
       [780.,  inf, 700., 460.],
       [320., 700.,  inf, 380.],
       [580., 460., 380.,  inf]])

In [17]:
maxi = np.inf
tree = Tree()
node_id = 1

%time Little(costs,0)

Wall time: 2 ms


In [18]:
tree.show()

Bordeaux
├── Bordeaux!Nantes 1820.0
│   ├── Bordeaux!Paris 1940.0
│   │   ├── Bordeaux!Lyon inf
│   │   └── Lyon 1940.0
│   │       ├── Lyon!Paris 2180.0
│   │       └── Paris 1940.0
│   │           ├── Nantes 1940.0
│   │           │   ├── Bordeaux 1940.0
│   │           │   └── Nantes!Bordeaux inf
│   │           └── Paris!Nantes inf
│   └── Paris 2060.0
└── Nantes 1820.0
    ├── Nantes!Paris 2060.0
    └── Paris 1940.0
        ├── Lyon 1940.0
        │   ├── Bordeaux 1940.0
        │   └── Lyon!Bordeaux inf
        └── Paris!Lyon inf



In [19]:
Path(tree)

['Bordeaux', 'Lyon', 'Paris', 'Nantes', 'Bordeaux']

# Integer Linear Programming

In [20]:
from pulp import *

In [21]:
nodes = [i for i in range(8)]

In [22]:
maxi = 1000
costs = np.array([[maxi,10,maxi,maxi,maxi,maxi,maxi,maxi],
                 [10,maxi,10,25,maxi,maxi,maxi,maxi],
                 [maxi,10,maxi,15,5,7,maxi,maxi],
                 [maxi,25,15,maxi,5,maxi,4,maxi],
                 [maxi,maxi,5,5,maxi,15,20,maxi],
                 [maxi,maxi,7,maxi,15,maxi,5,6],
                 [maxi,maxi,maxi,4,20,5,maxi,10],
                 [0,maxi,maxi,maxi,maxi,maxi,maxi,maxi]])

In [23]:
distances = dict(((i,j),costs[i,j]) for i in nodes for j in nodes if i!=j and costs[i,j] != maxi )

In [24]:
distances

{(0, 1): 10,
 (1, 0): 10,
 (1, 2): 10,
 (1, 3): 25,
 (2, 1): 10,
 (2, 3): 15,
 (2, 4): 5,
 (2, 5): 7,
 (3, 1): 25,
 (3, 2): 15,
 (3, 4): 5,
 (3, 6): 4,
 (4, 2): 5,
 (4, 3): 5,
 (4, 5): 15,
 (4, 6): 20,
 (5, 2): 7,
 (5, 4): 15,
 (5, 6): 5,
 (5, 7): 6,
 (6, 3): 4,
 (6, 4): 20,
 (6, 5): 5,
 (6, 7): 10,
 (7, 0): 0}

In [25]:
prob = LpProblem('TrackmaniaSalesmanProblem', LpMinimize)

In [26]:
x = LpVariable.dicts('x',distances, 0,1,LpBinary)

In [27]:
#the objective
cost = lpSum([x[(i,j)]*distances[(i,j)] for (i,j) in distances])
prob+=cost

In [28]:
#constraints
for k in nodes:
    #every site has exactly one inbound connection
    prob+= lpSum([ x[(i,k)] for i in nodes if (i,k) in x]) ==1
    #every site has exactly one outbound connection
    prob+=lpSum([ x[(k,i)] for i in nodes if (k,i) in x]) ==1

In [29]:
#we need to keep track of the order in the tour to eliminate the possibility of subtours
u = LpVariable.dicts('u', nodes, 0, len(nodes)-1, LpInteger)

In [30]:
#subtour elimination
N=len(nodes)
for i in nodes:
    for j in nodes:
        if i != j and (i != 0 and j!= 0) and (i,j) in x:
            prob += u[i] - u[j] <= (N)*(1-x[(i,j)]) - 1

In [31]:
%time prob.solve()
print(LpStatus[prob.status])

Wall time: 21 ms
Optimal


In [32]:
#Affichage du meilleur chemin
nodes_left = nodes.copy()
org = 0
tour=[]
tour.append(nodes_left.pop( nodes_left.index(org)))

while len(nodes_left) > 0:
    
    for k in nodes_left:
        try :
            if x[(org,k)].varValue ==1:
                tour.append( nodes_left.pop( nodes_left.index(k)))
                org=k
                break
        except:
            pass
        
tour.append(0)

tour_legs = [distances[(tour[i-1], tour[i])] for i in range(1,len(tour))]

print('Found optimal tour!')
tour_str = [str(tour[i]+1) for i in range(len(tour))]
print(' -> '.join(tour_str))

Found optimal tour!
1 -> 2 -> 3 -> 5 -> 4 -> 7 -> 6 -> 8 -> 1
